In [3]:
!pip install python-terrier
import pyterrier as pt
import numpy as np
pt.init()

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [4]:
import pandas as pd


In [3]:
import pandas as pd

dataset = pt.get_dataset("trec-deep-learning-passages")
train_topics = dataset.get_topics("train")
train_qrels = dataset.get_qrels("train")
train_topics = train_topics.sort_values(by='qid', ascending=True)[3:]
train_qrels = train_qrels.sort_values(by='qid', ascending=True)[3:]

# A simple join operation on the qids, as we have much more queries than qrels, we want to only use those
# queries with a qrel for optimal training performances. 
temp = pd.merge(train_topics, train_qrels, left_on='qid', right_on='qid')
# Make the new train_topics dataframe with only those queries that have a qrel
train_topics_100 = pd.concat([temp['qid'], temp['query']], axis=1, keys=['qid', 'query'])
# print(train_topics_100.head(100))

08:35:55.349 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


In [13]:
# Drop duplicates and only get the first 100 queries (with qrels), save those as a csv
# train_topics_100.drop_duplicates(inplace=True)
# train_topics_100 = train_topics_100[:100]
# train_topics_100.to_csv("train_topics_100.csv", index=False)

In [15]:
train_topics_100 = pd.read_csv("train_topics_100.csv")

In [16]:
# Get the qrels for the 100 queries that we have selected, and save as a csv
# train_qrel_100 = pd.concat([temp['qid'], temp['docno'], temp['label']], axis=1, keys=['qid', 'docno', 'label'])
# train_qrel_100.head(100)
# train_qrel_100.to_csv("train_qrel_100.csv", index=False)

In [18]:
train_qrel_100 = pd.read_csv("train_qrel_100.csv")

In [25]:
# Load the entire corpus of the MS Marco dataset
corpus = './corpus/collection.tsv'

In [35]:
# Put all documents in a dataframe and lowercase the passages
# all_documents = pd.read_csv(corpus, sep='\t', names=("doc_id","passage"))
# all_documents['passage'] = documents['passage'].apply(lambda x: x.lower())
# all_documents.to_csv("all_documents_corpus.csv", index="False")

In [39]:
all_documents = pd.read_csv("all_documents_corpus.csv")

In [40]:
glove_file = "./glove.6B.50d.txt"

In [44]:
def load_glove_model():
    glove_model = {}
    with open(glove_file,'r',encoding='utf8') as f:
        print("Execution here")
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            glove_model[word] = embedding
    print(f"{len(glove_model)} words loaded!")
    return glove_model

In [45]:
model = load_glove_model()

Execution here
400000 words loaded!


In [48]:
#Note to self - check length of the vector if it is 50 for Glove

# Function returning vector reperesentation of a document
def get_embedding(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(50)
    else:
        for word in doc_tokens:
            #if token exists in the trained model's vocabulary then fetch its vector
            try:
                embeddings.append(model[word])
            #if it is an out-of-vocabulary word, generate a random vector of len 300
            except:
                embeddings.append(np.random.rand(50))
            

    # Note to self - understand why we are computing mean of these vectors        
    # mean the vectors of individual words to get the vector of the document
    return np.mean(embeddings, axis=0)

In [49]:
import time

In [60]:
# documents['vector'] = documents['passage'].apply(lambda x :get_embedding(x.split()))

In [61]:
# documents.to_csv("passage_vectors_glove50d.csv", index=False)

In [62]:
# train_topics_100['vector'] = train_topics_100['query'].apply(lambda x :get_embedding(x.split()))

In [63]:
# train_topics_100.to_csv("queries_vectors_glove50d.csv", index=False)

In [5]:
documents = pd.read_csv("passage_vectors_glove50d.csv")
train_topics_100 = pd.read_csv("queries_vectors_glove50d.csv")

In [6]:
from scipy import spatial

In [13]:
docs_1000 = documents.iloc[:1000]

In [23]:
type(documents.iloc[23]['vector'])

str

In [14]:
import time

In [15]:
final_df = pd.DataFrame(columns=['qID', 'passageID', 'cSim'])
start = time.time()    
for idx1, query in train_topics_100.iterrows():
    qid = query['qid']
    q_vec = query['vector']
    for idx2, passage in docs_1000.iterrows():
        pid = passage['doc_id']
        p_vec = passage['vector']
        sim = 1 - spatial.distance.cosine(q_vec, p_vec)
            
        final_df = final_df.append({'qID': qid, 'passageID': pid, 'cSim': sim}, ignore_index=True)
            
final_df.to_csv('test.csv', index=False)
end = time.time()
print('took: ', end-start, ' time')

0it [02:42, ?it/s]


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U813'), dtype('<U813')) -> dtype('<U813')